In [1]:
!pip install folium==0.2.1
!pip install cdlib
!pip install leidenalg
!pip install imgaug==0.2.5
!pip install python-igraph
!pip install angel-cd

     |████████████████████████████████| 71kB 2.0MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79979 sha256=13d4288cda2d970e876d19fd433920ecbdf9f72de2c923771d5a689d40cd79fd
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 122kB 2.8MB/s 
     |████████████████████████████████| 13.6MB 302kB/s 
     |████████████████████████████████| 1.8MB 38.8MB/s 
     |████████████████████████████████| 1.9MB 34.1MB/s 
     |████████████████████████████████| 6.4MB 40.2MB/s 
     |████████████████████████████████| 1.8MB 41.5MB/s 
  Created wheel for pulp: filename=PuLP-1.6.10-cp36-none-any.whl size=12269903 sha256=67817d5aed2bd8a406bcaadceb803b30d10fc751c45b1b391db77fdb13039010
  Stored in directory: /root/.cache/pip/wheels/5e/76/77/e

In [2]:
import pickle
import math
import igraph
import pandas as pd
import numpy as np
from igraph import *
import matplotlib.pyplot as plt
import time
import cdlib

Reading graph

In [5]:
def load_obj(name):
    with open('/content/drive/My Drive/NetworkScience/AcademicGraph/data/obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name):
    with open('/content/drive/My Drive/NetworkScience/AcademicGraph/data/obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

g = load_obj("graph")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Extracting component

In [6]:
components = g.clusters(mode="weak")
biggerSubG = components.subgraph(22) # 22 = Bigger connected component index in components list

## Applying CD algorithms

In [21]:
from cdlib import algorithms
from cdlib import *

### Label propagation

In [8]:
lab = algorithms.label_propagation(biggerSubG)

### Louvain

In [9]:
louv = algorithms.louvain(biggerSubG)

### K-clique
k = 3

In [10]:
kCli = algorithms.kclique(biggerSubG, 3)

KeyboardInterrupt: ignored

### Angel

In [71]:
ang = algorithms.angel(biggerSubG, min_com_size=6, threshold=0.30)

TypeError: ignored

## Evaluation

In [11]:
# Discovered communities
coms = [lab, louv]

In [72]:
from cdlib import evaluation
from cdlib.utils import convert_graph_formats
import networkx as nx

In [69]:
intEvals = {}
nxG = convert_graph_formats(biggerSubG, nx.Graph, directed=None)

for com in coms:
  intEvals[com] = {}

for com in coms:
  intEvals[com]['intEdgeDensity'] = evaluation.internal_edge_density(nxG,com).score
  intEvals[com]['avgNodeDegree'] = evaluation.average_internal_degree(nxG,com).score
  intEvals[com]['modularity'] = evaluation.modularity_density(nxG,com).score
  intEvals[com]['conductance'] = evaluation.conductance(nxG,com).score

In [70]:
df = pd.DataFrame(intEvals)
df.columns = ['Labels', 'Louvains']
# df['Labels'] = pd.Series([m.score for m in df['Labels']])
#df['Louvains'] = pd.Series([m.score for m in df['Louvains']])
#df.fillna(0, inplace=True)
print(df)

#print(pd.Series([m.score for m in df['Labels']]))

                     Labels   Louvains
intEdgeDensity     0.187263   0.082292
avgNodeDegree      5.384196  78.163699
modularity      1106.914217  10.011709
conductance        0.245343   0.058672


## Comparison
Comparing n F1 score among partitions